## Genético n-Reinas

- 13/01/2023
- Algoritmos bioinspirados
- Aliaksandra Skrypko

In [10]:
def evaluaFO(tablero):
    nReinas = len(tablero)
    fit = 0
    for j in range(nReinas-1):
        for k in range(nReinas-j-1):
            if ((tablero[j+k+1]==tablero[j]+k+1) or (tablero[j+k+1]==tablero[j]-k-1)):
                fit=fit+1

    return(fit)


In [11]:
def cruce(individuo1, individuo2,permuSize):
  
    # cruce de orden
    
    descendientes = np.zeros([permuSize,2])
    descendientes = descendientes-1
  
    pos = sorted(random.sample(range(permuSize),2))
    descendientes[pos[0]:pos[1]+1,0] = individuo1[pos[0]:pos[1]+1]
    descendientes[pos[0]:pos[1]+1,1] = individuo2[pos[0]:pos[1]+1]

    #construyo descendiente 1  
    if (pos[1] < permuSize-1):
        k = pos[1]+1
        for j in range(pos[1]+1,permuSize):
            if (individuo2[j] not in descendientes[:,0]):
                descendientes[k,0] = individuo2[j]
                k = k+1
                if (k==permuSize):
                    k = 0                   
        for j in range(pos[1]+1):
            if (individuo2[j] not in descendientes[:,0]):
                descendientes[k,0] = individuo2[j]
                k = k+1
                if (k==permuSize):
                    k = 0                
    else:
        k = 0
        for j in range(pos[1]+1):
            if (individuo2[j] not in descendientes[:,0]):
                descendientes[k,0] = individuo2[j]
                k = k+1
                if (k==permuSize):
                    k = 0

    #construyo descendiente 2  
    if (pos[1] < permuSize-1):
        k = pos[1]+1
        for j in range(pos[1]+1,permuSize):
            if (individuo1[j] not in descendientes[:,1]):
                descendientes[k,1] = individuo1[j]
                k = k+1
                if (k==permuSize):
                    k = 0                   
        for j in range(pos[1]+1):
            if (individuo1[j] not in descendientes[:,1]):
                descendientes[k,1] = individuo1[j]
                k = k+1
                if (k==permuSize):
                    k = 0                
    else:
        k = 0
        for j in range(pos[1]+1):
            if (individuo1[j] not in descendientes[:,1]):
                descendientes[k,1] = individuo1[j]
                k = k+1
                if (k==permuSize):
                    k = 0
                
    return(descendientes)

In [12]:
def mutacion(hijos,pm,permuSize):
    # Mutación por intercambio
    for i in range(2):
        a = np.random.uniform()
        if(a < pm):
            pos = random.sample(range(permuSize),2)
            k1 = hijos[ pos[0] ,i]
            hijos [ pos[0] ,i] = hijos [ pos[1] ,i]
            hijos [ pos[1] ,i] = k1
    return(hijos)


In [13]:
def supervivientes(poblacion,fitness,matrizHijos,fitnessHijos,popSize,permuSize,numPadres):
    # Selección elitista: los mejores
    generacionActual = np.zeros([popSize+numPadres,permuSize])
    generacionActual[0:popSize,:] = poblacion
    generacionActual[(popSize):(popSize+numPadres),:] = matrizHijos

    fitnessGeneracion = np.zeros(2*popSize)

    fitnessGeneracion[0:popSize] = fitness
    fitnessGeneracion[popSize:2*popSize] = fitnessHijos
    
    orden = np.argsort(fitnessGeneracion) 
    
    supervivientes = np.zeros([popSize,permuSize+1])
    supervivientes[:,0:permuSize]=generacionActual[orden[0:popSize],:]
    supervivientes[:,permuSize]=fitnessGeneracion[orden[0:popSize]]
    
    return(supervivientes)

In [14]:
def pintaTablero(individuo):
    nReinas = len(individuo)
    tt = np.zeros([nReinas,nReinas])
    for i in range(nReinas):
        tt[i,int(individuo[i])]=1
        
    print(tt)  

In [15]:
import numpy as np
import random

def geneticoLeticia(permuSize,popSize,numPadres,pm,numIteraciones):
    if (numPadres%2==1):
        numPadres = numPadres-1

    #Poblacion inicial

    poblacion = np.zeros([popSize,permuSize])
    fitness = np.zeros(popSize)
    
    for i in range(popSize):
        poblacion[i,] = np.random.permutation(permuSize)
        fitness[i] = evaluaFO(poblacion[i,])
  
    #bucle principal. Criterio de parada: número de iteraciones
    for jj in range(numIteraciones):
        matrizHijos = np.zeros([numPadres,permuSize])
        
        for kk in range(int(numPadres/2)):
            
            #Seleccion de padres torneo (cojo 2 y me quedo con el mejor, dos veces)
            quienes = random.sample(range(popSize),2)
            v = np.argsort(fitness[quienes]) # Me ordena de menor a mayor y me devuelve los índices
            u = v[0] # Me interesa el menor, o sea, el primero
            candidato1 = quienes[u] 
            
            quienes = random.sample(range(popSize),2)
            v = np.argsort(fitness[quienes]) # Me ordena de menor a mayor y me devuelve los índices
            u = v[0] # Me interesa el menor, o sea, el último
            candidato2 = quienes[u]

            #Crossover
            hijos = cruce(poblacion[candidato1,],poblacion[candidato2,],permuSize)
    
            #mutacion
            hijos = mutacion(hijos,pm,permuSize)
            hijos = np.transpose(hijos)

            #Guardo los hijos
            matrizHijos[(kk*2):(kk*2+2),:] = hijos
        
        #Evaluacion de hijos
        fitnessHijos = np.zeros(numPadres)
    
        for i in range(numPadres):
            fitnessHijos[i] = evaluaFO(matrizHijos[i,])
     
        #Seleccion supervivientes
        res = supervivientes(poblacion,fitness,matrizHijos,fitnessHijos,popSize,permuSize,numPadres)
        poblacion = res[:,0:permuSize]
        fitness = res[:,permuSize]
            
        #imprime info
        print("Iteración ",jj,": min = ",min(fitness), ", max = ", max(fitness),", media = ",np.mean(fitness))
        if(min(fitness)==0):
            break
      
   
  
    #Resultado final
    i = np.argmin(fitness)
    if(fitness[i]==0):
        pintaTablero(poblacion[i,:])
  
  
        
        
        

In [16]:
geneticoLeticia(4,10,10,0.1,10)

Iteración  0 : min =  0.0 , max =  1.0 , media =  0.7
[[0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]]
